# Installs

In [ ]:
!pip install pydub
!pip install gradio
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size

In [ ]:
!pip install speechbrain
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Imports

In [ ]:
import os

In [ ]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio
import torch

In [ ]:
import librosa
import numpy as np

# from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio as sisnr
# from torchmetrics.functional.audio import signal_distortion_ratio as sdr

from torchmetrics.audio import ScaleInvariantSignalNoiseRatio as sisnr
from torchmetrics.audio import SignalDistortionRatio as sdr

from itertools import permutations

from speechbrain.nnet.losses import PitWrapper

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# Make Librimix

In [ ]:
!wget -c --tries=0 --read-timeout=20 http://www.openslr.org/resources/12/test-clean.tar.gz -P "/"
!tar -xzf "test-clean.tar.gz" -C "/content/"
!rm -rf test-clean.tar.gz

--2024-04-11 17:14:35--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2024-04-11 17:14:42--  http://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

est-clean.tar.gz    100%[===================>] 330.60M  20.0MB/s    in 18s     

2024-04-11 17:15:02 (18.7 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [ ]:
!wget -c --tries=0 --read-timeout=20 https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip -P "/"
!unzip -qn wham_noise.zip -d "/content/"
!rm -rf wham_noise.zip

--2024-04-11 17:15:06--  https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip
Resolving my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)... 3.5.28.57, 52.217.234.185, 54.231.204.233, ...
Connecting to my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)|3.5.28.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18194930748 (17G) [application/zip]
Saving to: ‘wham_noise.zip’

ham_noise.zip       100%[===================>]  16.95G  23.1MB/s    in 11m 57s 

2024-04-11 17:27:03 (24.2 MB/s) - ‘wham_noise.zip’ saved [18194930748/18194930748]



In [ ]:
!git clone https://github.com/JorisCos/LibriMix
!pip install -r LibriMix/requirements.txt

Cloning into 'LibriMix'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 274 (delta 71), reused 58 (delta 58), pack-reused 163
Receiving objects: 100% (274/274), 33.43 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (116/116), done.
  Preparing metadata (setup.py) ... done
  Created wheel for pysndfx: filename=pysndfx-0.3.6-py3-none-any.whl size=6539 sha256=0397fb98efb2c2a5bd89619d262a7f1ce363aab9528b5c3a26acd49d27634523
  Stored in directory: /root/.cache/pip/wheels/e8/73/04/275af1c6a789bbc12f388bce2d5353e77579e3aaadd36c61fe
Successfully built pysndfx


In [ ]:
dir_path = "/content/LibriMix/metadata/Libri2Mix"

for file in os.listdir(dir_path):
    if "libri2mix_test-clean" not in file:
        !rm {dir_path + "/" + file}


In [ ]:
metadata_dir = "LibriMix/metadata/Libri2Mix"

#!mkdir librimix
!python LibriMix/scripts/create_librimix_from_metadata.py --librispeech_dir "LibriSpeech" \
    --wham_dir "/content/wham_noise" \
    --metadata_dir {metadata_dir} \
    --librimix_outdir "librimix" \
    --n_src 2 \
    --freqs 8k \
    --modes max \
    --types mix_clean mix_single

Directory librimix/Libri2Mix/wav8k/max/test already exist. Files won't be overwritten


In [ ]:
!rm -r wham_noise
!rm -r LibriSpeech

In [ ]:
import pandas as pd

csv_path1 = "/content/librimix/Libri2Mix/wav8k/max/metadata/metrics_test_mix_clean.csv"
csv_path2 = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"

dataframe1 = pd.read_csv(csv_path1)
dataframe2 = pd.read_csv(csv_path2)

# Task 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def load_csv_data(csv_path):
    return pd.read_csv(csv_path)

def split_dataset(data, train_size=0.7, random_state=42):
    train_set, test_set = train_test_split(data, train_size=train_size, random_state=random_state)
    return train_set, test_set

librimix_csv_path = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"
librimix_csv = load_csv_data(librimix_csv_path)
train_set, test_set = split_dataset(librimix_csv)

In [ ]:
def load_and_initialize_model(model_source, save_directory, device_name):
    # Assuming `separator` is an object or module that has been previously defined and imported.
    # This object should have a method `from_hparams` capable of loading the model.
    model = separator.from_hparams(source=model_source, savedir=save_directory, run_opts={"device": device_name})
    model = model.to(device_name)  # Ensure the model is moved to the specified device.
    return model

model_source = "speechbrain/sepformer-whamr"
save_directory = "pretrained_models/sepformer-whamr"

# Load and initialize the model
model = load_and_initialize_model(model_source, save_directory, device_name)

hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

In [ ]:
def load_audio(path):
    y , sr = librosa.load(path, sr = 8000)
    return torch.tensor(librosa.util.normalize(y)).unsqueeze(0)

In [ ]:
class AudioMetrics:
    def __init__(self, **kwargs):
        self.metric_sisnr = PermutationInvariantTraining(scale_invariant_signal_noise_ratio, mode="speaker-wise", eval_func='max')
        self.metric_sdr = PermutationInvariantTraining(signal_distortion_ratio, mode="speaker-wise", eval_func='max')

    def preprocess(self, input_signal, estimated_signal, reference_signal):
        input_signal_ = torch.concat([input_signal, input_signal], dim=2).to(reference_signal.device)

        estimated_signal = estimated_signal.permute((0, 2, 1))
        reference_signal = reference_signal.permute((0, 2, 1))
        input_signal_ = input_signal_.permute((0, 2, 1))
        return input_signal_, estimated_signal, reference_signal

    def compute_sisnr(self, input_signal, estimated_signal, reference_signal):
        self.metric_sisnr.to(reference_signal.device)
        input_signal, estimated_signal, reference_signal = self.preprocess(input_signal, estimated_signal, reference_signal)

        sisnr_estimated = self.metric_sisnr(estimated_signal, reference_signal)
        sisnr_input = self.metric_sisnr(input_signal, reference_signal)

        sisnr_improvement = sisnr_estimated - sisnr_input

        return sisnr_improvement

    def compute_sdr(self, input_signal, estimated_signal, reference_signal):
        self.metric_sdr.to(reference_signal.device)
        input_signal, estimated_signal, reference_signal = self.preprocess(input_signal, estimated_signal, reference_signal)

        sdr_estimated = self.metric_sdr(estimated_signal, reference_signal)
        sdr_input = self.metric_sdr(input_signal, reference_signal)

        sdr_improvement = sdr_estimated - sdr_input

        return sdr_improvement

In [ ]:
import numpy as np
import torch
from tqdm.notebook import tqdm

class ModelEvaluator:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        # Initialize the SI-SNR and SDR improvement calculation functions
        self.metric = AudioMetrics()

    def load_audio(self, path):
        y , sr = librosa.load(path, sr = 8000)
        return torch.tensor(librosa.util.normalize(y)).unsqueeze(0)

    def evaluate_model(self, csv):
        sisnr = []
        sdr = []

        for _, row in tqdm(list(csv.iterrows())):
            sr1 = self.load_audio(row['source_1_path'])
            sr2 = self.load_audio(row['source_2_path'])
            mix = self.load_audio(row['mixture_path'])

            sr1 = sr1.to(self.device)
            sr2 = sr2.to(self.device)
            mix = mix.to(self.device)

            with torch.no_grad():
                output = self.model(mix)

            target = torch.stack([sr1, sr2], dim=2)
            mix = mix.unsqueeze(-1)

            sisnr.append((self.metric.compute_sisnr(mix, output, target)).detach().cpu().tolist())
            sdr.append((self.metric.compute_sdr(mix, output, target)).detach().cpu().tolist())

        return np.mean(sisnr), np.mean(sdr)

In [ ]:
Evaluator = ModelEvaluator(model, device)

In [ ]:
sisnri_score , sdri_score = Evaluator.evaluate_model(model , test_set)

print("Scores obtained")
print(f"\t SI-SNRi :: {sisnri_score}")
print(f"\t SDri    :: {sdri_score}")

  0%|          | 0/900 [00:00<?, ?it/s]

Score obtained on test dataset using Sepformer is as follows
	 SI-SNRi :: 8.52612478699949
	 SDri    :: 11.615369231535329


In [3]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import copy
import torch
from torch.cuda.amp import GradScaler
from torch.optim import Adam
from tqdm.notebook import tqdm
from speechbrain.nnet.losses import get_si_snr_with_pitwrapper as sisnr_loss

class AudioModelTrainer:
    def __init__(self, model, train_set, device, epochs=1, learning_rate=15e-5, clip_grad_norm=5, loss_upper_lim=999999):
        self.model = copy.deepcopy(model)
        self.train_set = train_set
        self.device = device
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.clip_grad_norm = clip_grad_norm
        self.loss_upper_lim = loss_upper_lim
        self.scaler = GradScaler()
        self.optimizer = Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = sisnr_loss
        self.model.train()
        for param in self.model.parameters():
            param.requires_grad = True
        wandb.init(project="Q2_training", config={
            "learning_rate": self.learning_rate,
            "epochs": self.epochs,
        })

    def train_epoch(self):
        sisnri = []
        sdri = []
        loss = []
        metric = AudioMetrics()
        nonfinite_count = 0

        with torch.cuda.amp.autocast(dtype=torch.float16, device_type=self.device.type):
            pbar = tqdm(list(self.train_set.iterrows()))
            for _, row in pbar:
                sr1, sr2, mix = self.load_and_preprocess(row)
                self.optimizer.zero_grad(set_to_none=True)
                output = self.model(mix)
                target = torch.stack([sr1, sr2], dim=2)
                loss_value = self.compute_loss(output, target)
                if loss_value is not None:
                    self.backward_and_step(loss_value)
                else:
                    nonfinite_count += 1
                mix = mix.unsqueeze(-1)
                loss.append(loss_value.detach().cpu().tolist() if loss_value is not None else 0)
                sisnri.append(sisnri_func(mix, output, target).detach().cpu().tolist())
                sdri.append(sdri_func(mix, output, target).detach().cpu().tolist())
        return np.mean(loss), np.mean(sisnri), np.mean(sdri)

    def load_and_preprocess(self, row):
        sr1 = load_audio(row['source_1_path'])[:, :80000].to(self.device)
        sr2 = load_audio(row['source_2_path'])[:, :80000].to(self.device)
        mix = load_audio(row['mixture_path'])[:, :80000].to(self.device)
        return sr1, sr2, mix

    def compute_loss(self, output, target):
        loss_value = self.criterion(output, target)
        loss_value = loss_value[loss_value > -30]
        if loss_value.nelement() > 0:
            loss_value = loss_value.mean()
            if loss_value < self.loss_upper_lim:
                return loss_value
        return None

    def backward_and_step(self, loss_value):
        self.scaler.scale(loss_value).backward()
        if self.clip_grad_norm >= 0:
            self.scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_grad_norm)
        self.scaler.step(self.optimizer)
        self.scaler.update()

    def train(self):
        for epoch in range(self.epochs):
            epoch_loss, epoch_sisnri, epoch_sdri = self.train_epoch()
            print(f"EPOCH {epoch} -->")
            print(f"\tloss    :: {epoch_loss}")
            print(f"\tSI-SNRi :: {epoch_sisnri}")
            print(f"\tSDRi    :: {epoch_sdri}")
            wandb.log({"loss": epoch_loss, "SI-SNRi": epoch_sisnri, "SDRi": epoch_sdri})
        wandb.finish()
        self.optimizer.zero_grad(set_to_none=True)

model_trainer = AudioModelTrainer(model, train_set, device)
model_trainer.train()

EPOCH,▁█
SDRi,█▁
SI-SNRi,█▁
loss,▁█
EPOCH,1
SDRi,3.51145
SI-SNRi,2.34447
loss,-2.34674


In [ ]:
sisnri_score , sdri_score = Evaluator.evaluate_model(model , test_set)

print("Scores obtained")
print(f"\t SI-SNRi :: {sisnri_score}")
print(f"\t SDri    :: {sdri_score}")

  0%|          | 0/900 [00:00<?, ?it/s]

Score obtained on test dataset using finetuned Sepformer is as follows
	 SI-SNRi :: -6.765212313466602
	 SDri    :: -5.014605642888281


In [ ]:
import gradio as gr
import torch
import librosa

class EnhancedSepFormer:
    def __init__(self, trained_model):
        self.trained_model = trained_model

    def process_audio(self, input_audio):
        sample_rate, audio_data = input_audio
        mixed_audio = self.prepare_audio(audio_data).to(self.trained_model.device)

        with torch.no_grad():
            separated_audio = self.trained_model(mixed_audio)

        separated_channel_1 = separated_audio[:, :, 0].detach().cpu().tolist()
        separated_channel_2 = separated_audio[:, :, 1].detach().cpu().tolist()

        return (sample_rate, separated_channel_1), (sample_rate, separated_channel_2)

    def prepare_audio(self, audio_signal):
        normalized_audio = librosa.util.normalize(audio_signal)
        return torch.tensor(normalized_audio).unsqueeze(0)

# Setting up the Gradio interface
interface = gr.Interface(
    fn=EnhancedSepFormer(finetuned_model),
    inputs=gr.Audio(label="Upload Mixed Audio"),
    outputs=["audio", "audio"],
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://602d0bebd56409fb94.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
